In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Ensure that we always get the same results
np.random.seed(0)

# Helper function to generate a random data sample
def generate_random_data_sample(sample_size, feature_dim, num_classes):
    # Create synthetic data using NumPy.
    Y = np.random.randint(size=(sample_size, 1), low=0, high=num_classes)
    # Make sure that the data is separable
    X = (np.random.randn(sample_size, feature_dim)+5) * (Y+1)
    return X, Y

In [ ]:
# Generate random separable data
input_dim = 2
num_output_classes = 2
sample_size = 128
x, labels = generate_random_data_sample(sample_size, input_dim, num_output_classes)
print(labels.shape)
colors = ['#333333' if label == 0 else '#b7b7b7' for label in labels]

# generate random decision boundary
theta = np.random.randn(3)
x_values = [np.min(x[:, 0] - 5), np.max(x[:, 1] + 5)]
y_values = -(theta[0] + np.dot(theta[1], x_values))/theta[2]

plt.figure(figsize=(10,5))
ax = plt.gca()
ax.grid(color='#b7b7b7', linestyle='-', linewidth=0.5, alpha=0.5)
plt.scatter(x[:,0], x[:,1], c=colors)
plt.plot(x_values, y_values, label='Decision Boundary', color='#121212', linewidth=1, alpha=0.9)
plt.show()

## Logistic regression

Optimize decision boundary with logistic regression.

### 1. Feedforward
\begin{equation}
z = W^TX
\end{equation}

\begin{equation}
\hat{y} = \sigma(z)
\end{equation}

\begin{equation}
\sigma(z) = \frac{1}{1+e^{-z}}
\end{equation}

### 2. Compute cost function
\begin{equation}
Loss(y, \hat{y}) = -\frac{1}{n}(y^Tlog(\hat{y})+(1-y)^Tlog(1-\hat{y}))
\end{equation}

### 3. Backpropagation
\begin{eqnarray}
\frac{\delta Loss(y, \hat{y})}{\delta W} &=& \frac{\delta Loss(y, \hat{y})}{\delta \hat{y}}\cdot\frac{\delta \hat{y}}{\delta z}\cdot\frac{\delta z}{\delta W} \\
&=& \frac{1}{n}X^T(\hat{y}-y)
\end{eqnarray}

### 4. Gradient descent
\begin{equation}
W = W - \alpha  \frac{\delta Loss(y, \hat{y})}{\delta W}
\end{equation}

In [ ]:
class LogisticRegression:
    def __init__(self, x, y, iterations, learning_rate):
        self.input       = np.c_[np.ones((x.shape[0], 1)), x]
        self.y           = y
        self._iterations = iterations
        self._rate       = learning_rate
        self.weights     = np.random.rand(self.input.shape[1],1) 
        self.output      = np.zeros(y.shape)
        
    def _sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))
    
    def _feedforward(self):
        self.output = self._sigmoid(self.input.dot(self.weights))
        
    def _backprop(self):
        delta_weights = self.input.T.dot(self.output-self.y)/len(self.y)        
        self.weights -= self._rate * delta_weights
    
    def get_loss(self):
        return -np.asscalar(self.y.T.dot(np.log(self.output))+(1-self.y).T.dot(np.log(1-self.output)))/len(self.y)

    def train(self):
        for i in range(self._iterations):
            self._feedforward()
            self._backprop()
            
iterations = 1000
learning_rate = 0.2
y = np.array([labels.T[0]]).T

lr = LogisticRegression(x, y, iterations, learning_rate)
lr.train()
loss = lr.get_loss()

new_y_values = -(lr.weights[0] + lr.weights[1]* x_values)/lr.weights[2]
plt.figure(figsize=(10,5))
ax = plt.gca()
ax.grid(color='#b7b7b7', linestyle='-', linewidth=0.5, alpha=0.5)
ax.text(0, 0, f'error = {loss:.2f}',fontsize=12,color='#000000')
plt.scatter(x[:,0], x[:,1], c=colors)
plt.plot(x_values, new_y_values, label='Decision Boundary', color='#121212', linewidth=2, alpha=0.9)
plt.show()